# Python

![](https://leukipp.github.io/xmastree/kernelspecs/python.png)

In [ ]:
import os
import js
import csv
import urllib
import numpy as np

from pyodide_js import FS
from IPython.display import HTML, Javascript, display


## Utils

In [ ]:
class FileSystemUtils(object):
    def __init__(self, fs, path):
        """ indexedDB filesystem storage """
        self.fs = fs
        self.path = path
        self.type = self.fs.filesystems.IDBFS

        self.mount()
        self.upsync()

    def mount(self):
        """ mount kernel filesystem """
        if not os.path.exists(self.path):
            self.fs.mkdir(self.path)
            self.fs.mount(self.type, {}, self.path)

    def upsync(self):
        """ sync from browser to kernel filesystem """
        self.fs.syncfs(True, lambda x: print(x) if x else None)

    def downsync(self):
        """ sync from kernel to browser filesystem """
        self.fs.syncfs(False, lambda x: print(x) if x else None)


class Simulator(object):
    def __init__(self, width='100%', height='600px'):
        """ three js simulator """
        self.width = width
        self.height = height
        self.settings = dict()
        self.rendered = False

        self.id = 'simulator'
        self.root = '' if '127.0.0.1' in str(js.location) else '/xmastree'
        self.url = f'{self.root}/files/simulator/index.html'

    def src(self):
        """ source url for iframe """
        return f'{self.url}#{urllib.parse.urlencode(self.settings)}'

    def render(self, **kwargs):
        """ render simulator html """
        self.rendered = False
        self.update(**kwargs)
        self.rendered = True
        layout = {
            'id': self.id,
            'frameborder': '0',
            'width': self.width,
            'height': self.height,
            'src': self.src()
        }
        attributes = ' '.join([f'{k}="{v}"' for k, v in layout.items()])
        display(HTML(f'<iframe allowfullscreen {attributes} />'))

    def update(self, **kwargs):
        """ update simulator settings """
        for k, v in kwargs.items():
            self.settings[k] = str(v)
        self.settings['seed'] = int(np.random.rand() * 1e6)
        if self.rendered:
            display(Javascript(f'document.getElementById("{self.id}").src="{self.src()}"'))


class Frames(list):
    def __init__(self, coords_csv, fps=30):
        """ animation frames helper """
        coords = list(csv.reader(coords_csv.split('\n')))
        self.coords = np.array([[float(col) for col in row] for row in coords])
        self.size = self.coords.shape[0]
        self.fps = fps

    def export(self, name, fs):
        """ export frames as csv """
        head = ['FRAME_ID'] + np.hstack([[f'R_{i}', f'G_{i}', f'B_{i}'] for i in range(self.size)]).tolist()
        rows = np.array(self, dtype=int)

        ids = np.arange(rows.shape[0]).reshape(-1, 1)
        frames = np.reshape(rows, (*rows.shape[:1], -1))
        data = np.hstack([ids, frames])

        path = os.path.join(fs.path, name)
        np.savetxt(path, data, header=','.join(head), fmt='%i', delimiter=',', comments='')
        fs.downsync()

        return path


## Setup

In [ ]:
# init filesystem
fs = FileSystemUtils(FS, path=os.path.join(os.path.expanduser('~'), 'data'))

# init simulator
simulator = Simulator()

# init frames
response = await js.fetch('https://raw.githubusercontent.com/standupmaths/xmastree2021/main/coords_2021.csv')
coords = await response.text()
frames = Frames(coords, fps=30)


## Code

In [ ]:
# clear frames
frames.clear()

# ------------------------------------------ START CODING ------------------------------------------

# write frames
for frame_id in range(90):
    frame = np.zeros(shape=(frames.size, 3))

    # assign random colors
    for led_id in range(frames.size):
        rgb = np.random.randint(0, 256, size=(1, 3))
        frame[led_id] = rgb

    frames.append(frame)

# ------------------------------------------- END CODING -------------------------------------------

# export frames
path = frames.export('animation.csv', fs)

# update simulator
simulator.update(fps=frames.fps, animations=f'fs:/{path}')


In [ ]:
# render simulator
simulator.render(loop='true', rotation=0)
